In [ ]:
# experimento com todas as imagens das Palmer para predição do tempo de cada amostra, usando CV 5

In [1]:
import cv2
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import math
import scipy

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

%load_ext autoreload
plt.rcParams['figure.figsize'] = [20, 15]

path = os.getcwd()+'\\..\\imagens\\'
sys.path.append(os.getcwd()+'\\..\\scripts\\')
from MNG import MNG
from MNGFeatures import MNGFeatures
from MNGFeaturesMeans import MNGFeaturesMeans
from MNGFeaturesSize import MNGFeaturesSize
from MNGModel import MNGModel

In [2]:
mean_size_data = pd.read_csv(path+'\\..\\features\\means_size_all_half.csv', sep=';', index_col=0)

In [17]:
indexes = ['fold1', 'fold2', 'fold3', 'fold4', 'fold5']
columns = ['MLR', 'RF100', 'RF200', 'RF300', 'RF400', 'RF500', 'RF600', 'RF700']

cv_num = 5

In [18]:
# mean and size

mean_size_res_r2 = pd.DataFrame(index=indexes, columns=columns)
mean_size_res_rmse = pd.DataFrame(index=indexes, columns=columns)
X = mean_size_data.drop(columns='time')
Y = mean_size_data['time']

mlr_model = LinearRegression()
r2_scores = cross_val_score(mlr_model, X, Y, scoring='r2', cv=cv_num)
mse_scores = cross_val_score(mlr_model, X, Y, scoring='neg_mean_squared_error', cv=cv_num)
mean_size_res_r2['MLR'] = r2_scores
mean_size_res_rmse['MLR'] = mse_scores

for trees in np.arange(100, 701, 100):
    rf_model = RandomForestRegressor(n_estimators=trees)
    r2_scores = cross_val_score(rf_model, X, Y, scoring='r2', cv=cv_num)
    mse_scores = cross_val_score(rf_model, X, Y, scoring='neg_mean_squared_error', cv=cv_num)
        
    mean_size_res_r2['RF'+str(trees)] = r2_scores
    mean_size_res_rmse['RF'+str(trees)] = mse_scores

In [20]:
cross_val_score

<function sklearn.model_selection._validation.cross_val_score(estimator, X, y=None, groups=None, scoring=None, cv=None, n_jobs=1, verbose=0, fit_params=None, pre_dispatch='2*n_jobs')>

In [14]:
mean_size_res_r2

,MLR,RF100,RF200,RF300,RF400,RF500,RF600,RF700
fold1,-103.175755,-4.521463,-4.546778,-4.543097,-4.538102,-4.530835,-4.555629,-4.537980
fold2,-1.162595,-2.315037,-2.377472,-2.293089,-2.480491,-2.367135,-2.485287,-2.377191
fold3,-1.481023,-1.355524,-1.264627,-1.343689,-1.322152,-1.247080,-1.245626,-1.277789
fold4,-1.375675,-1.770372,-1.748425,-1.765316,-1.729525,-1.751477,-1.773851,-1.695022
fold5,-7.752111,-14.272888,-13.485114,-13.690222,-13.577838,-13.950123,-14.051066,-13.823397


In [21]:
mean_size_res_rmse

,MLR,RF100,RF200,RF300,RF400,RF500,RF600,RF700
fold1,-9685.762310,-513.967159,-514.099347,-514.151894,-515.398924,-515.150918,-515.442647,-515.921514
fold2,-249.324012,-432.092311,-400.302405,-380.088681,-402.392391,-398.986916,-393.804585,-391.269121
fold3,-304.489195,-286.626326,-282.789659,-278.244937,-288.306876,-282.473315,-283.512736,-278.649507
fold4,-273.889778,-320.026269,-307.019347,-312.996197,-313.387667,-314.081207,-312.695044,-310.606153
fold5,-913.185141,-1517.467670,-1554.121037,-1592.559318,-1523.896843,-1551.915602,-1545.605952,-1561.995944
